In [12]:
#import packages
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go
import math
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from dateutil import parser
import requests
from bs4 import BeautifulSoup as Soup
from datetime import date, timedelta
import datetime 
import dateutil.relativedelta
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [13]:
#load files
#game data
df_1617_gt = pd.read_csv('nba1617_gt.csv')  
df_1718_gt = pd.read_csv('nba1718_gt.csv')
frames = [df_1617_gt,df_1718_gt]
df_all_games = pd.concat(frames)

#viewership data
df_ILP = pd.read_csv('International_League_Pass_Viewership.csv')

#social data
df_gt = pd.read_csv('NBA_gts_161718.csv')
df_wp = pd.read_csv('nba_wp_pageviews.csv')
df_social = pd.read_csv('New_Social_Media_Metrics.csv')

In [14]:
#wikipedia page views
#convert to datetime
df_wp['date'] = pd.to_datetime(df_wp.date)

#extract month and year
df_wp['month'] = df_wp['date'].dt.month
df_wp['year'] = df_wp['date'].dt.year

#sum wikipedia pageviews
df_wp = df_wp.groupby(['team', 'month', 'year'])['wp_pageviews'].sum().reset_index()

In [15]:
#google trends

#sort date ascending
df_gt = df_gt.sort_values('date', ascending = True)

#convert to datetime
df_gt['date'] = pd.to_datetime(df_gt.date)

#extract month and year
df_gt['month'] = df_gt['date'].dt.month
df_gt['year'] = df_gt['date'].dt.year

#average of google trends scores in that month
df_gt = df_gt.groupby(['team', 'month', 'year'])['gts'].mean().reset_index()



In [16]:
#sum web metric score over a period of one month

#load team web data
df_web = pd.read_csv('Updated_Team_Web_Metrics.csv')

#"UNQ" = Total unique viewers of team website properties on the given Date; 
#"TTS" = Total time spent on team website properties on the given Date, in second
df_web['DTime'] = pd.to_datetime(df_web.Date) #converting to date to datetime
df_web['Date'] = df_web['DTime'].dt.date #converting to date to datetime

df_web = df_web.sort_values('Date',ascending=True)

#UNQ and TTS
df_web_unq = df_web[df_web['Metric']=='UNQ']
df_web_tts = df_web[df_web['Metric']=='TTS']

#make unq and tts columns
df_web_unq=df_web_unq.rename(columns = {'Value':'UNQ'})
df_web_tts=df_web_tts.rename(columns = {'Value':'TTS'})

#updated to contain columns for UNQ and TTS separately
df_web_updated = pd.merge(df_web_tts[['Team','Date','DTime','TTS']],df_web_unq[['Team','Date','UNQ']],on=['Date','Team'],how='left')


#filter web data for 2016-17 
df_web_updated = df_web_updated[df_web_updated.DTime >= '2016-05-01' ] 



#convert to date time
df_web_updated['date'] = pd.to_datetime(df_web_updated.Date)  

#extract month and year
df_web_updated['month'] = df_web_updated['date'].dt.month
df_web_updated['year'] = df_web_updated['date'].dt.year

print(len(df_web_updated))
#sum web metric pageviews
df_web_TTS = df_web_updated.groupby(['Team', 'month', 'year'])['TTS'].sum().reset_index()
df_web_UNQ = df_web_updated.groupby(['Team', 'month', 'year'])['UNQ'].mean().reset_index()


#
df_web_updated = pd.merge(df_web_TTS, df_web_UNQ, on=['Team', 'month', 'year'], how='left')




24780
840


In [18]:
#team abbreviation -> team name
team_abbrev_dict = {'New York Knicks':'NYK', 
                    'Utah Jazz':'UTA', 
                    'San Antonio Spurs':'SAS', 
                    'Miami Heat':'MIA', 
                    'Dallas Mavericks':'DAL', 
                    'Brooklyn Nets':'BRK', 
                    'Detroit Pistons':'DET', 
                    'Charlotte Hornets':'CHA', 
                    'Minnesota Timberwolves':'MIN',
                    'Oklahoma City Thunder':'OKC', 
                    'Sacramento Kings':'SAC', 
                    'Houston Rockets':'HOU', 
                    'Washington Wizards':'WAS', 
                    'Boston Celtics':'BOS', 
                    'Los Angeles Clippers':'LAC', 
                    'Cleveland Cavaliers':'CLE', 
                    'Indiana Pacers':'IND', 
                    'Orlando Magic':'ORL',
                    'Phoenix Suns':'PHX', 
                    'Los Angeles Lakers':'LAL', 
                    'Golden State Warriors':'GSW', 
                    'Memphis Grizzlies':'MEM', 
                    'New Orleans Pelicans':'NOP', 
                    'Portland Trail Blazers':'POR', 
                    'Milwaukee Bucks':'MIL', 
                    'Denver Nuggets':'DEN', 
                    'Chicago Bulls':'CHI',
                    'Toronto Raptors':'TOR', 
                    'Philadelphia 76ers':'PHI', 
                    'Atlanta Hawks':'ATL'
}

def map_abbr_team(team):
    """Maps team abbreviation to actual name"""
    team = team_abbrev_dict[team]
    return team




In [19]:
#map teams to team names and locations
df_all_games['Away'] = df_all_games['Away'].apply(lambda x:map_abbr_team(x))
df_all_games['Home'] = df_all_games['Home'].apply(lambda x:map_abbr_team(x))
df_gt['team'] = df_gt['team'].apply(lambda x:map_abbr_team(x))
df_wp['team'] = df_wp['team'].apply(lambda x:map_abbr_team(x))


In [20]:


#finds previous month and extract year for game data
df_all_games['Date'] = pd.to_datetime(df_all_games['Date'])
dateutil.relativedelta.relativedelta(months=-1)
df_all_games['prev_month'] = (df_all_games['Date'] - pd.DateOffset(months=1)).dt.month #previous month
df_all_games['year'] = (df_all_games['Date'] - pd.DateOffset(months=1)).dt.year #year of previous month
df_all_games['prev_month_2'] = (df_all_games['Date'] - pd.DateOffset(months=2)).dt.month #2 months ago
df_all_games['year_2'] = (df_all_games['Date'] - pd.DateOffset(months=2)).dt.year #year of 2 months ago
df_all_games['prev_month_3'] = (df_all_games['Date'] - pd.DateOffset(months=3)).dt.month #3 months ago
df_all_games['year_3'] = (df_all_games['Date'] - pd.DateOffset(months=3)).dt.year #year of 3 months ago
df_all_games['prev_month_4'] = (df_all_games['Date'] - pd.DateOffset(months=4)).dt.month #4 months ago
df_all_games['year_4'] = (df_all_games['Date'] - pd.DateOffset(months=4)).dt.year #4 months ago

In [21]:
#change column names for merging purposes
#social data
df_social = df_social.rename(columns = {'Month':'prev_month'})
df_social = df_social.rename(columns = {'Year':'year'})
df_social['Home'] = df_social['Team']
df_social['Away'] = df_social['Team']
df_social['prev_month_2'] = df_social['prev_month'] 
df_social['year_2'] = df_social['year'] 
df_social['prev_month_3'] = df_social['prev_month'] 
df_social['year_3'] = df_social['year'] 
df_social['prev_month_4'] = df_social['prev_month'] 
df_social['year_4'] = df_social['year'] 

#google trends
df_gt = df_gt.rename(columns = {'month':'prev_month'})
df_gt['Home'] = df_gt['team']
df_gt['Away'] = df_gt['team']
df_gt['prev_month_2'] = df_gt['prev_month'] 
df_gt['year_2'] = df_gt['year'] 
df_gt['prev_month_3'] = df_gt['prev_month'] 
df_gt['year_3'] = df_gt['year'] 
df_gt['prev_month_4'] = df_gt['prev_month'] 
df_gt['year_4'] = df_gt['year'] 

#wikipedia page views
df_wp = df_wp.rename(columns = {'month':'prev_month'})
df_wp['Home'] = df_wp['team']
df_wp['Away'] = df_wp['team']
df_wp['prev_month_2'] = df_wp['prev_month'] 
df_wp['year_2'] = df_wp['year'] 
df_wp['prev_month_3'] = df_wp['prev_month'] 
df_wp['year_3'] = df_wp['year'] 
df_wp['prev_month_4'] = df_wp['prev_month'] 
df_wp['year_4'] = df_wp['year'] 

#web metric
df_web_updated = df_web_updated.rename(columns = {'month':'prev_month'})
df_web_updated['Home'] = df_web_updated['Team']
df_web_updated['Away'] = df_web_updated['Team']
df_web_updated['prev_month_2'] = df_web_updated['prev_month'] 
df_web_updated['year_2'] = df_web_updated['year'] 
df_web_updated['prev_month_3'] = df_web_updated['prev_month'] 
df_web_updated['year_3'] = df_web_updated['year'] 
df_web_updated['prev_month_4'] = df_web_updated['prev_month'] 
df_web_updated['year_4'] = df_web_updated['year'] 

In [22]:

#map google trends  to specific team and date range
#home
#1 month
df_all_games = pd.merge(df_all_games,df_gt.drop(['Away','team','prev_month_2', 'year_2', 'prev_month_3', 'year_3', 'prev_month_4', 'year_4'],axis=1), on = ['year','prev_month','Home'], how='left')
df_all_games=df_all_games.rename(columns = {'gts':'Home_gts_1'})

#2 months
df_all_games = pd.merge(df_all_games,df_gt.drop(['Away','team', 'prev_month', 'year', 'prev_month_3', 'year_3', 'prev_month_4', 'year_4'],axis=1), on = ['year_2','prev_month_2','Home'], how='left')
df_all_games=df_all_games.rename(columns = {'gts':'Home_gts_2'})

#3 months
df_all_games = pd.merge(df_all_games,df_gt.drop(['Away','team', 'prev_month_2', 'year_2', 'prev_month', 'year', 'prev_month_4', 'year_4'],axis=1), on = ['year_3','prev_month_3','Home'], how='left')
df_all_games=df_all_games.rename(columns = {'gts':'Home_gts_3'})

#4 months
df_all_games = pd.merge(df_all_games,df_gt.drop(['Away','team', 'prev_month_2', 'year_2', 'prev_month_3', 'year_3', 'prev_month', 'year'],axis=1), on = ['year_4','prev_month_4','Home'], how='left')
df_all_games=df_all_games.rename(columns = {'gts':'Home_gts_4'})



#away
#1 month
df_all_games = pd.merge(df_all_games,df_gt.drop(['Home','team','prev_month_2', 'year_2', 'prev_month_3', 'year_3', 'prev_month_4', 'year_4'],axis=1), on = ['year','prev_month','Away'], how='left')
df_all_games=df_all_games.rename(columns = {'gts':'Away_gts_1'})

#2 months
df_all_games = pd.merge(df_all_games,df_gt.drop(['Home','team', 'prev_month', 'year', 'prev_month_3', 'year_3', 'prev_month_4', 'year_4'],axis=1), on = ['year_2','prev_month_2','Away'], how='left')
df_all_games=df_all_games.rename(columns = {'gts':'Away_gts_2'})

#3 months
df_all_games = pd.merge(df_all_games,df_gt.drop(['Home','team', 'prev_month_2', 'year_2', 'prev_month', 'year', 'prev_month_4', 'year_4'],axis=1), on = ['year_3','prev_month_3','Away'], how='left')
df_all_games=df_all_games.rename(columns = {'gts':'Away_gts_3'})

#4 months
df_all_games = pd.merge(df_all_games,df_gt.drop(['Home','team', 'prev_month_2', 'year_2', 'prev_month_3', 'year_3', 'prev_month', 'year'],axis=1), on = ['year_4','prev_month_4','Away'], how='left')
df_all_games=df_all_games.rename(columns = {'gts':'Away_gts_4'})


In [23]:

#map wikipedia page views to specific team and date range
#home
#1 month
df_all_games = pd.merge(df_all_games,df_wp.drop(['Away','team','prev_month_2', 'year_2', 'prev_month_3', 'year_3', 'prev_month_4', 'year_4'],axis=1), on = ['year','prev_month','Home'], how='left')
df_all_games=df_all_games.rename(columns = {'wp_pageviews':'Home_wp_pageviews_1'})

#2 months
df_all_games = pd.merge(df_all_games,df_wp.drop(['Away','team', 'prev_month', 'year', 'prev_month_3', 'year_3', 'prev_month_4', 'year_4'],axis=1), on = ['year_2','prev_month_2','Home'], how='left')
df_all_games=df_all_games.rename(columns = {'wp_pageviews':'Home_wp_pageviews_2'})

#3 months
df_all_games = pd.merge(df_all_games,df_wp.drop(['Away','team', 'prev_month_2', 'year_2', 'prev_month', 'year', 'prev_month_4', 'year_4'],axis=1), on = ['year_3','prev_month_3','Home'], how='left')
df_all_games=df_all_games.rename(columns = {'wp_pageviews':'Home_wp_pageviews_3'})

#4 months
df_all_games = pd.merge(df_all_games,df_wp.drop(['Away','team', 'prev_month_2', 'year_2', 'prev_month_3', 'year_3', 'prev_month', 'year'],axis=1), on = ['year_4','prev_month_4','Home'], how='left')
df_all_games=df_all_games.rename(columns = {'wp_pageviews':'Home_wp_pageviews_4'})



#away
#1 month
df_all_games = pd.merge(df_all_games,df_wp.drop(['Home','team','prev_month_2', 'year_2', 'prev_month_3', 'year_3', 'prev_month_4', 'year_4'],axis=1), on = ['year','prev_month','Away'], how='left')
df_all_games=df_all_games.rename(columns = {'wp_pageviews':'Away_wp_pageviews_1'})

#2 months
df_all_games = pd.merge(df_all_games,df_wp.drop(['Home','team', 'prev_month', 'year', 'prev_month_3', 'year_3', 'prev_month_4', 'year_4'],axis=1), on = ['year_2','prev_month_2','Away'], how='left')
df_all_games=df_all_games.rename(columns = {'wp_pageviews':'Away_wp_pageviews_2'})

#3 months
df_all_games = pd.merge(df_all_games,df_wp.drop(['Home','team', 'prev_month_2', 'year_2', 'prev_month', 'year', 'prev_month_4', 'year_4'],axis=1), on = ['year_3','prev_month_3','Away'], how='left')
df_all_games=df_all_games.rename(columns = {'wp_pageviews':'Away_wp_pageviews_3'})

#4 months
df_all_games = pd.merge(df_all_games,df_wp.drop(['Home','team', 'prev_month_2', 'year_2', 'prev_month_3', 'year_3', 'prev_month', 'year'],axis=1), on = ['year_4','prev_month_4','Away'], how='left')
df_all_games=df_all_games.rename(columns = {'wp_pageviews':'Away_wp_pageviews_4'})


In [24]:

#map social data to specific team and date range
#home
df_all_games = pd.merge(df_all_games,df_social.drop(['Away','Team','prev_month_2', 'year_2', 'prev_month_3', 'year_3', 'prev_month_4', 'year_4'],axis=1), on = ['year','prev_month','Home'], how='left')
df_all_games=df_all_games.rename(columns = {'Followers_Facebook':'Home_Followers_Facebook_1'})
df_all_games=df_all_games.rename(columns = {'Followers_Twitter':'Home_Followers_Twitter_1'})
df_all_games=df_all_games.rename(columns = {'Followers_Instagram':'Home_Followers_Instagram_1'})
df_all_games=df_all_games.rename(columns = {'Followers_Snapchat':'Home_Followers_Snapchat_1'})
df_all_games=df_all_games.rename(columns = {'Followers_Weibo':'Home_Followers_Weibo_1'})
df_all_games=df_all_games.rename(columns = {'Engagements_Facebook':'Home_Engagements_Facebook_1'})
df_all_games=df_all_games.rename(columns = {'Engagements_Twitter':'Home_Engagements_Twitter_1'})
df_all_games=df_all_games.rename(columns = {'Engagements_Instagram':'Home_Engagements_Instagram_1'})
df_all_games=df_all_games.rename(columns = {'Impressions_Facebook':'Home_Impressions_Facebook_1'})
df_all_games=df_all_games.rename(columns = {'Impressions_Twitter':'Home_Impressions_Twitter_1'})

df_all_games = pd.merge(df_all_games,df_social.drop(['Away','Team','prev_month', 'year', 'prev_month_3', 'year_3', 'prev_month_4', 'year_4'],axis=1), on = ['year_2','prev_month_2','Home'], how='left')
df_all_games=df_all_games.rename(columns = {'Followers_Facebook':'Home_Followers_Facebook_2'})
df_all_games=df_all_games.rename(columns = {'Followers_Twitter':'Home_Followers_Twitter_2'})
df_all_games=df_all_games.rename(columns = {'Followers_Instagram':'Home_Followers_Instagram_2'})
df_all_games=df_all_games.rename(columns = {'Followers_Snapchat':'Home_Followers_Snapchat_2'})
df_all_games=df_all_games.rename(columns = {'Followers_Weibo':'Home_Followers_Weibo_2'})
df_all_games=df_all_games.rename(columns = {'Engagements_Facebook':'Home_Engagements_Facebook_2'})
df_all_games=df_all_games.rename(columns = {'Engagements_Twitter':'Home_Engagements_Twitter_2'})
df_all_games=df_all_games.rename(columns = {'Engagements_Instagram':'Home_Engagements_Instagram_2'})
df_all_games=df_all_games.rename(columns = {'Impressions_Facebook':'Home_Impressions_Facebook_2'})
df_all_games=df_all_games.rename(columns = {'Impressions_Twitter':'Home_Impressions_Twitter_2'})

df_all_games = pd.merge(df_all_games,df_social.drop(['Away','Team','prev_month_2', 'year_2', 'prev_month', 'year', 'prev_month_4', 'year_4'],axis=1), on = ['year_3','prev_month_3','Home'], how='left')
df_all_games=df_all_games.rename(columns = {'Followers_Facebook':'Home_Followers_Facebook_3'})
df_all_games=df_all_games.rename(columns = {'Followers_Twitter':'Home_Followers_Twitter_3'})
df_all_games=df_all_games.rename(columns = {'Followers_Instagram':'Home_Followers_Instagram_3'})
df_all_games=df_all_games.rename(columns = {'Followers_Snapchat':'Home_Followers_Snapchat_3'})
df_all_games=df_all_games.rename(columns = {'Followers_Weibo':'Home_Followers_Weibo_3'})
df_all_games=df_all_games.rename(columns = {'Engagements_Facebook':'Home_Engagements_Facebook_3'})
df_all_games=df_all_games.rename(columns = {'Engagements_Twitter':'Home_Engagements_Twitter_3'})
df_all_games=df_all_games.rename(columns = {'Engagements_Instagram':'Home_Engagements_Instagram_3'})
df_all_games=df_all_games.rename(columns = {'Impressions_Facebook':'Home_Impressions_Facebook_3'})
df_all_games=df_all_games.rename(columns = {'Impressions_Twitter':'Home_Impressions_Twitter_3'})

df_all_games = pd.merge(df_all_games,df_social.drop(['Away','Team','prev_month_2', 'year_2', 'prev_month_3', 'year_3', 'prev_month', 'year'],axis=1), on = ['year_4','prev_month_4','Home'], how='left')
df_all_games=df_all_games.rename(columns = {'Followers_Facebook':'Home_Followers_Facebook_4'})
df_all_games=df_all_games.rename(columns = {'Followers_Twitter':'Home_Followers_Twitter_4'})
df_all_games=df_all_games.rename(columns = {'Followers_Instagram':'Home_Followers_Instagram_4'})
df_all_games=df_all_games.rename(columns = {'Followers_Snapchat':'Home_Followers_Snapchat_4'})
df_all_games=df_all_games.rename(columns = {'Followers_Weibo':'Home_Followers_Weibo_4'})
df_all_games=df_all_games.rename(columns = {'Engagements_Facebook':'Home_Engagements_Facebook_4'})
df_all_games=df_all_games.rename(columns = {'Engagements_Twitter':'Home_Engagements_Twitter_4'})
df_all_games=df_all_games.rename(columns = {'Engagements_Instagram':'Home_Engagements_Instagram_4'})
df_all_games=df_all_games.rename(columns = {'Impressions_Facebook':'Home_Impressions_Facebook_4'})
df_all_games=df_all_games.rename(columns = {'Impressions_Twitter':'Home_Impressions_Twitter_4'})


#away
df_all_games = pd.merge(df_all_games,df_social.drop(['Home','Team','prev_month_2', 'year_2', 'prev_month_3', 'year_3', 'prev_month_4', 'year_4'],axis=1), on = ['year','prev_month','Away'], how='left')
df_all_games=df_all_games.rename(columns = {'Followers_Facebook':'Away_Followers_Facebook_1'})
df_all_games=df_all_games.rename(columns = {'Followers_Twitter':'Away_Followers_Twitter_1'})
df_all_games=df_all_games.rename(columns = {'Followers_Instagram':'Away_Followers_Instagram_1'})
df_all_games=df_all_games.rename(columns = {'Followers_Snapchat':'Away_Followers_Snapchat_1'})
df_all_games=df_all_games.rename(columns = {'Followers_Weibo':'Away_Followers_Weibo_1'})
df_all_games=df_all_games.rename(columns = {'Engagements_Facebook':'Away_Engagements_Facebook_1'})
df_all_games=df_all_games.rename(columns = {'Engagements_Twitter':'Away_Engagements_Twitter_1'})
df_all_games=df_all_games.rename(columns = {'Engagements_Instagram':'Away_Engagements_Instagram_1'})
df_all_games=df_all_games.rename(columns = {'Impressions_Facebook':'Away_Impressions_Facebook_1'})
df_all_games=df_all_games.rename(columns = {'Impressions_Twitter':'Away_Impressions_Twitter_1'})

df_all_games = pd.merge(df_all_games,df_social.drop(['Home','Team','prev_month', 'year', 'prev_month_3', 'year_3', 'prev_month_4', 'year_4'],axis=1), on = ['year_2','prev_month_2','Away'], how='left')
df_all_games=df_all_games.rename(columns = {'Followers_Facebook':'Away_Followers_Facebook_2'})
df_all_games=df_all_games.rename(columns = {'Followers_Twitter':'Away_Followers_Twitter_2'})
df_all_games=df_all_games.rename(columns = {'Followers_Instagram':'Away_Followers_Instagram_2'})
df_all_games=df_all_games.rename(columns = {'Followers_Snapchat':'Away_Followers_Snapchat_2'})
df_all_games=df_all_games.rename(columns = {'Followers_Weibo':'Away_Followers_Weibo_2'})
df_all_games=df_all_games.rename(columns = {'Engagements_Facebook':'Away_Engagements_Facebook_2'})
df_all_games=df_all_games.rename(columns = {'Engagements_Twitter':'Away_Engagements_Twitter_2'})
df_all_games=df_all_games.rename(columns = {'Engagements_Instagram':'Away_Engagements_Instagram_2'})
df_all_games=df_all_games.rename(columns = {'Impressions_Facebook':'Away_Impressions_Facebook_2'})
df_all_games=df_all_games.rename(columns = {'Impressions_Twitter':'Away_Impressions_Twitter_2'})

df_all_games = pd.merge(df_all_games,df_social.drop(['Home','Team','prev_month_2', 'year_2', 'prev_month', 'year', 'prev_month_4', 'year_4'],axis=1), on = ['year_3','prev_month_3','Away'], how='left')
df_all_games=df_all_games.rename(columns = {'Followers_Facebook':'Away_Followers_Facebook_3'})
df_all_games=df_all_games.rename(columns = {'Followers_Twitter':'Away_Followers_Twitter_3'})
df_all_games=df_all_games.rename(columns = {'Followers_Instagram':'Away_Followers_Instagram_3'})
df_all_games=df_all_games.rename(columns = {'Followers_Snapchat':'Away_Followers_Snapchat_3'})
df_all_games=df_all_games.rename(columns = {'Followers_Weibo':'Away_Followers_Weibo_3'})
df_all_games=df_all_games.rename(columns = {'Engagements_Facebook':'Away_Engagements_Facebook_3'})
df_all_games=df_all_games.rename(columns = {'Engagements_Twitter':'Away_Engagements_Twitter_3'})
df_all_games=df_all_games.rename(columns = {'Engagements_Instagram':'Away_Engagements_Instagram_3'})
df_all_games=df_all_games.rename(columns = {'Impressions_Facebook':'Away_Impressions_Facebook_3'})
df_all_games=df_all_games.rename(columns = {'Impressions_Twitter':'Away_Impressions_Twitter_3'})

df_all_games = pd.merge(df_all_games,df_social.drop(['Home','Team','prev_month_2', 'year_2', 'prev_month_3', 'year_3', 'prev_month', 'year'],axis=1), on = ['year_4','prev_month_4','Away'], how='left')
df_all_games=df_all_games.rename(columns = {'Followers_Facebook':'Away_Followers_Facebook_4'})
df_all_games=df_all_games.rename(columns = {'Followers_Twitter':'Away_Followers_Twitter_4'})
df_all_games=df_all_games.rename(columns = {'Followers_Instagram':'Away_Followers_Instagram_4'})
df_all_games=df_all_games.rename(columns = {'Followers_Snapchat':'Away_Followers_Snapchat_4'})
df_all_games=df_all_games.rename(columns = {'Followers_Weibo':'Away_Followers_Weibo_4'})
df_all_games=df_all_games.rename(columns = {'Engagements_Facebook':'Away_Engagements_Facebook_4'})
df_all_games=df_all_games.rename(columns = {'Engagements_Twitter':'Away_Engagements_Twitter_4'})
df_all_games=df_all_games.rename(columns = {'Engagements_Instagram':'Away_Engagements_Instagram_4'})
df_all_games=df_all_games.rename(columns = {'Impressions_Facebook':'Away_Impressions_Facebook_4'})
df_all_games=df_all_games.rename(columns = {'Impressions_Twitter':'Away_Impressions_Twitter_4'})



In [25]:
#map wikipedia page views to specific team and date range
#home
#1 month
df_all_games = pd.merge(df_all_games,df_web_updated.drop(['Away','Team','prev_month_2', 'year_2', 'prev_month_3', 'year_3', 'prev_month_4', 'year_4'],axis=1), on = ['year','prev_month','Home'], how='left')
df_all_games=df_all_games.rename(columns = {'TTS':'Home_TTS_1'})
df_all_games=df_all_games.rename(columns = {'UNQ':'Home_UNQ_1'})

#2 months
df_all_games = pd.merge(df_all_games,df_web_updated.drop(['Away','Team', 'prev_month', 'year', 'prev_month_3', 'year_3', 'prev_month_4', 'year_4'],axis=1), on = ['year_2','prev_month_2','Home'], how='left')
df_all_games=df_all_games.rename(columns = {'TTS':'Home_TTS_2'})
df_all_games=df_all_games.rename(columns = {'UNQ':'Home_UNQ_2'})

#3 months
df_all_games = pd.merge(df_all_games,df_web_updated.drop(['Away','Team', 'prev_month_2', 'year_2', 'prev_month', 'year', 'prev_month_4', 'year_4'],axis=1), on = ['year_3','prev_month_3','Home'], how='left')
df_all_games=df_all_games.rename(columns = {'TTS':'Home_TTS_3'})
df_all_games=df_all_games.rename(columns = {'UNQ':'Home_UNQ_3'})

#4 months
df_all_games = pd.merge(df_all_games,df_web_updated.drop(['Away','Team', 'prev_month_2', 'year_2', 'prev_month_3', 'year_3', 'prev_month', 'year'],axis=1), on = ['year_4','prev_month_4','Home'], how='left')
df_all_games=df_all_games.rename(columns = {'TTS':'Home_TTS_4'})
df_all_games=df_all_games.rename(columns = {'UNQ':'Home_UNQ_4'})


#away
#1 month
df_all_games = pd.merge(df_all_games,df_web_updated.drop(['Home','Team','prev_month_2', 'year_2', 'prev_month_3', 'year_3', 'prev_month_4', 'year_4'],axis=1), on = ['year','prev_month','Away'], how='left')
df_all_games=df_all_games.rename(columns = {'TTS':'Away_TTS_1'})
df_all_games=df_all_games.rename(columns = {'UNQ':'Away_UNQ_1'})

#2 months
df_all_games = pd.merge(df_all_games,df_web_updated.drop(['Home','Team', 'prev_month', 'year', 'prev_month_3', 'year_3', 'prev_month_4', 'year_4'],axis=1), on = ['year_2','prev_month_2','Away'], how='left')
df_all_games=df_all_games.rename(columns = {'TTS':'Away_TTS_2'})
df_all_games=df_all_games.rename(columns = {'UNQ':'Away_UNQ_2'})

#3 months
df_all_games = pd.merge(df_all_games,df_web_updated.drop(['Home','Team', 'prev_month_2', 'year_2', 'prev_month', 'year', 'prev_month_4', 'year_4'],axis=1), on = ['year_3','prev_month_3','Away'], how='left')
df_all_games=df_all_games.rename(columns = {'TTS':'Away_TTS_3'})
df_all_games=df_all_games.rename(columns = {'UNQ':'Away_UNQ_3'})

#4 months
df_all_games = pd.merge(df_all_games,df_web_updated.drop(['Home','Team', 'prev_month_2', 'year_2', 'prev_month_3', 'year_3', 'prev_month', 'year'],axis=1), on = ['year_4','prev_month_4','Away'], how='left')
df_all_games=df_all_games.rename(columns = {'TTS':'Away_TTS_4'})
df_all_games=df_all_games.rename(columns = {'UNQ':'Away_UNQ_4'})

In [26]:
#export final social data
df_all_games.to_csv('social.csv')

In [27]:
# All-star games are included in the original dataset(USA vs. World, West vs. East)
# Preseason games against teams abroad are included ( RMD-Real Madrid, FCB - FC Barcelona,MAC - Maccabi Haifa,SDS -  Shanghai Sharks,SLA - Buenos Aires San Lorenzo )
# Remove these games and only include actual NBA games
df_ILP = pd.read_csv('Updated_International_League_Pass_Viewership.csv')
df_ILP = df_ILP[~df_ILP.Home.isin(['USA','WST','RMD','FCB'])]
df_ILP = df_ILP[~df_ILP.Away.isin(['WLD','EST','MAC','SDS','SLA'])]

#convert date string to datetime
df_ILP.Date = pd.to_datetime(df_ILP.Date)

#create Game column across all 3 datasets
df_ILP['Game'] = df_ILP.Home+'-'+df_ILP.Away+'-'+df_ILP.Date.astype(str)
df_all_games['Game'] = df_all_games.Home+'-'+df_all_games.Away+'-'+df_all_games.Date.astype(str)


In [28]:
###identifies playoff games 
#2017-18 playoff dates 2018-04-14 to 2018-06-17
#2016-17 playoff dates 2017-04-15 to 2017-06-12
#NBA league pass does not include the finals.
df_all_games['Playoff'] = np.where(((df_all_games.Date >= '2018-04-14') & (df_all_games.Date <= '2018-06-17')) | ((df_all_games.Date >= '2017-04-15') & (df_all_games.Date <= '2017-06-12')),1,0) 

###identifies preseason games
#2017-18 preseason dates 2017-10-01 to 2017-10-12
#2016-17 preseason dates 2016-10-01 to 2016-10-20
df_all_games['Pre_Season'] = np.where(((df_all_games.Date >= '2017-10-01') & (df_all_games.Date <= '2017-10-12')) | ((df_all_games.Date >= '2016-10-01') & (df_all_games.Date <= '2016-10-23')),1,0) 


In [29]:
#total number of unique viewers and minutes watched for each game
df_ILP = df_ILP.groupby(['Game','Eastern_Time_Hr'])['Unique_Viewers','Minutes_Watched'].sum().reset_index()
df_ILP['Minutes_per_Viewer'] = df_ILP['Minutes_Watched']/df_ILP['Unique_Viewers']
df_ILP = df_ILP[df_ILP['Minutes_per_Viewer']>0] #removing games with negative minutes

#merge to main dataset
df_all_games_updated = pd.merge(df_all_games, df_ILP.drop('Eastern_Time_Hr', axis=1), on='Game', how='left')
df_all_games_updated = df_all_games_updated[df_all_games_updated.Unique_Viewers>0] #remove games that we don't have viewership data for

In [30]:
#secondary transactions
df_secondary = pd.read_csv('Secondary_Transactions_by_Game.csv')
df_secondary['Date'] = [str(x) for x in pd.to_datetime(df_secondary['Date']).dt.date.tolist()]
df_secondary['Game'] = df_secondary['Home'] + '-' + df_secondary['Away'] + '-' + df_secondary['Date']
df_secondary['Avg_Markup_Over_Full_STM_Price'] = [float(x.strip('%')) for x in df_secondary.Avg_Markup_Over_Full_STM_Price.tolist()]
df_secondary['Avg_Markup_Over_Full_STM_Price'] = df_secondary['Avg_Markup_Over_Full_STM_Price']/100 + 1 #change mark up percentage to numerical



#merge to main dataset
df_all_games_updated = pd.merge(df_all_games_updated, df_secondary[['Num_Secondary_Transactions','Avg_Markup_Over_Full_STM_Price','Game']], on='Game', how='left') 






In [31]:
#grabs relevant fields
df_all_games_updated = df_all_games_updated[['Date',
 'Start (ET)',
 'Home',
 'Home_Points',
 'Away',
 'Away_Points',
 'Home_gts_1',
 'Home_gts_2',
 'Home_gts_3',
 'Home_gts_4',
 'Away_gts_1',
 'Away_gts_2',
 'Away_gts_3',
 'Away_gts_4',
 'Home_wp_pageviews_1',
 'Home_wp_pageviews_2',
 'Home_wp_pageviews_3',
 'Home_wp_pageviews_4',
 'Away_wp_pageviews_1',
 'Away_wp_pageviews_2',
 'Away_wp_pageviews_3',
 'Away_wp_pageviews_4',
 'Home_Followers_Facebook_1',
 'Home_Followers_Twitter_1',
 'Home_Followers_Instagram_1',
 'Home_Followers_Snapchat_1',
 'Home_Followers_Weibo_1',
 'Home_Engagements_Facebook_1',
 'Home_Engagements_Twitter_1',
 'Home_Engagements_Instagram_1',
 'Home_Impressions_Facebook_1',
 'Home_Impressions_Twitter_1',
 'Home_Followers_Facebook_2',
 'Home_Followers_Twitter_2',
 'Home_Followers_Instagram_2',
 'Home_Followers_Snapchat_2',
 'Home_Followers_Weibo_2',
 'Home_Engagements_Facebook_2',
 'Home_Engagements_Twitter_2',
 'Home_Engagements_Instagram_2',
 'Home_Impressions_Facebook_2',
 'Home_Impressions_Twitter_2',
 'Home_Followers_Facebook_3',
 'Home_Followers_Twitter_3',
 'Home_Followers_Instagram_3',
 'Home_Followers_Snapchat_3',
 'Home_Followers_Weibo_3',
 'Home_Engagements_Facebook_3',
 'Home_Engagements_Twitter_3',
 'Home_Engagements_Instagram_3',
 'Home_Impressions_Facebook_3',
 'Home_Impressions_Twitter_3',
 'Home_Followers_Facebook_4',
 'Home_Followers_Twitter_4',
 'Home_Followers_Instagram_4',
 'Home_Followers_Snapchat_4',
 'Home_Followers_Weibo_4',
 'Home_Engagements_Facebook_4',
 'Home_Engagements_Twitter_4',
 'Home_Engagements_Instagram_4',
 'Home_Impressions_Facebook_4',
 'Home_Impressions_Twitter_4',
 'Away_Followers_Facebook_1',
 'Away_Followers_Twitter_1',
 'Away_Followers_Instagram_1',
 'Away_Followers_Snapchat_1',
 'Away_Followers_Weibo_1',
 'Away_Engagements_Facebook_1',
 'Away_Engagements_Twitter_1',
 'Away_Engagements_Instagram_1',
 'Away_Impressions_Facebook_1',
 'Away_Impressions_Twitter_1',
 'Away_Followers_Facebook_2',
 'Away_Followers_Twitter_2',
 'Away_Followers_Instagram_2',
 'Away_Followers_Snapchat_2',
 'Away_Followers_Weibo_2',
 'Away_Engagements_Facebook_2',
 'Away_Engagements_Twitter_2',
 'Away_Engagements_Instagram_2',
 'Away_Impressions_Facebook_2',
 'Away_Impressions_Twitter_2',
 'Away_Followers_Facebook_3',
 'Away_Followers_Twitter_3',
 'Away_Followers_Instagram_3',
 'Away_Followers_Snapchat_3',
 'Away_Followers_Weibo_3',
 'Away_Engagements_Facebook_3',
 'Away_Engagements_Twitter_3',
 'Away_Engagements_Instagram_3',
 'Away_Impressions_Facebook_3',
 'Away_Impressions_Twitter_3',
 'Away_Followers_Facebook_4',
 'Away_Followers_Twitter_4',
 'Away_Followers_Instagram_4',
 'Away_Followers_Snapchat_4',
 'Away_Followers_Weibo_4',
 'Away_Engagements_Facebook_4',
 'Away_Engagements_Twitter_4',
 'Away_Engagements_Instagram_4',
 'Away_Impressions_Facebook_4',
 'Away_Impressions_Twitter_4',
 'Home_TTS_1',
 'Home_UNQ_1',
 'Home_TTS_2',
 'Home_UNQ_2',
 'Home_TTS_3',
 'Home_UNQ_3',
 'Home_TTS_4',
 'Home_UNQ_4',
 'Away_TTS_1',
 'Away_UNQ_1',
 'Away_TTS_2',
 'Away_UNQ_2',
 'Away_TTS_3',
 'Away_UNQ_3',
 'Away_TTS_4',
 'Away_UNQ_4',
 'Game',
 'Playoff',
 'Pre_Season',
 'Unique_Viewers',
 'Minutes_Watched',
 'Minutes_per_Viewer',
 'Num_Secondary_Transactions',
 'Avg_Markup_Over_Full_STM_Price']]

In [32]:
#export dataframe
df_all_games_updated.to_csv('nba_main_data.csv', index=False)